SNIPPET 1: Setup & Configuration

-Install dependencies and load configuration file

MLflow, PyTorch, embeddings libraries, NLP tools

Load YAML config with all hyperparameters

Initialize MLflow tracking

In [ ]:
#!pip install mlflow pyyaml torch transformers datasets gensim scikit-learn pandas numpy matplotlib seaborn nltk -q

In [6]:
# Cell 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import yaml
import json
import mlflow
from mlflow import log_metric, log_param
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Ensure required NLTK resources are available. Try common locations and download if missing.
# Some environments may request 'punkt_tab' instead of 'punkt', so handle both.
_resource_map = {
    'punkt': 'tokenizers/punkt',
    'punkt_tab': 'tokenizers/punkt_tab',
    'punkt_tab': 'tokenizers/punkt_tab',
    'averaged_perceptron_tagger': 'taggers/averaged_perceptron_tagger'
}

for pkg, path in _resource_map.items():
    try:
        nltk.data.find(path)
    except LookupError:
        try:
            nltk.download(pkg, quiet=True)
        except Exception:
            # If download fails for any reason, continue — later tokenization will fall back
            pass
        
from collections import Counter
from gensim.models import Word2Vec, FastText
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
from sklearn.metrics import f1_score, hamming_loss, precision_score, recall_score

# Cell 3: Set Random Seeds
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ PyTorch device: {device}")

# Cell 4: Load Configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✓ Configuration loaded from config.yaml")
print(f"✓ Experiment: {config['mlflow']['experiment_name']}")

# Cell 5: Initialize MLflow
mlflow.set_tracking_uri(config['mlflow']['tracking_uri'])

experiment = mlflow.get_experiment_by_name(config['mlflow']['experiment_name'])
if experiment is None:
    experiment_id = mlflow.create_experiment(config['mlflow']['experiment_name'])
    print(f"✓ Created MLflow experiment (ID: {experiment_id})")
else:
    print(f"✓ Using existing MLflow experiment (ID: {experiment.experiment_id})")

mlflow.set_experiment(config['mlflow']['experiment_name'])

print("\n✓ SNIPPET 1 COMPLETE: Setup & Configuration Ready")


✓ PyTorch device: cpu
✓ Configuration loaded from config.yaml
✓ Experiment: MultiLabel_Text_Classification
✓ Using existing MLflow experiment (ID: 441613159991694359)

✓ SNIPPET 1 COMPLETE: Setup & Configuration Ready


In [6]:
# SNIPPET 2: Local CSV Dataset Loading & Preprocessing

print("\n" + "="*70)
print("STEP 1: LOCAL CSV DATASET LOADING & PREPROCESSING")
print("="*70)

# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import pandas as pd
import numpy as np
from collections import Counter
import re
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK data if needed
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

print("✓ All dependencies imported and NLTK ready")

# ============================================================================
# CELL 2: DEFINE GLOBAL CONFIG (No dependencies on external config)
# ============================================================================

# Define config dictionary with all necessary defaults
config = {
    "preprocessing": {
        "lowercase": True,
        "remove_urls": True,
        "remove_special_chars": True,
    },
    "data": {
        "train_size": None,      # Use full data
        "val_size": None,
        "test_size": None,
        "train_split": 0.8,      # 80% train
        "val_split": 0.1,        # 10% val
        "test_split": 0.1,       # 10% test
    },
    "num_classes": None,         # Will be auto-detected
    "random_seed": 42,
}

print("✓ Config initialized with defaults")
print(f"  - Train/Val/Test split: {config['data']['train_split']}/{config['data']['val_split']}/{config['data']['test_split']}")

# ============================================================================
# CELL 3: LOAD LOCAL CSV DATASET
# ============================================================================

print("\n" + "-"*70)
print("Loading local multilabel dataset from CSV...")
print("-"*70)

# Try common dataset paths
dataset_paths = [
    "./mLabel_tweets.csv",
    "./data/mLabel_tweets.csv",
    "./twitter_multilabel.csv",
    "./tweets.csv",
    "./dataset.csv",
    "mLabel_tweets.csv",
]

df = None
dataset_path = None

for path in dataset_paths:
    try:
        df = pd.read_csv(path)
        dataset_path = path
        print(f"\n✓ Successfully loaded: {path}")
        break
    except FileNotFoundError:
        continue
    except Exception as e:
        print(f"⚠ Warning loading {path}: {e}")
        continue

# If still not found, provide helpful message
if df is None:
    print(f"\n❌ ERROR: Could not find dataset at any of these locations:")
    for path in dataset_paths:
        print(f"    - {path}")
    print(f"\n📌 Please:")
    print(f"   1. Place your CSV file in one of the above locations, OR")
    print(f"   2. Modify the 'dataset_paths' list above with your file path")
    raise FileNotFoundError("Dataset CSV not found!")

# ============================================================================
# CELL 4: DATASET STRUCTURE INSPECTION
# ============================================================================

print(f"\nDataset Information:")
print(f"  Shape: {df.shape} (rows, columns)")
print(f"  Columns: {list(df.columns)}")
print(f"\nFirst 3 rows:")
print(df.head(3).to_string())

print(f"\nColumn data types:")
print(df.dtypes)

print(f"\nDataset info:")
print(f"  Missing values: {df.isnull().sum().sum()}")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# ============================================================================
# CELL 5: AUTO-DETECT TEXT AND LABEL COLUMNS
# ============================================================================

def detect_text_column(df):
    """Auto-detect which column contains text/tweet content"""
    
    text_keywords = ['text', 'tweet', 'content', 'message', 'review', 
                     'description', 'sentence', 'comment', 'post', 'caption', 
                     'title', 'body', 'doc', 'passage']
    
    # Look for exact match
    for col in df.columns:
        if col.lower() in text_keywords:
            return col
    
    # Look for partial match
    for col in df.columns:
        for keyword in text_keywords:
            if keyword in col.lower():
                return col
    
    # Use first object column with long strings
    for col in df.columns:
        if df[col].dtype == 'object':
            avg_length = df[col].astype(str).str.len().mean()
            if avg_length > 20:  # Likely text column
                return col
    
    return None

def detect_label_column(df, exclude_col=None):
    """Auto-detect which column(s) contain labels"""
    
    label_keywords = ['label', 'emotion', 'sentiment', 'category', 'tag', 
                      'class', 'target', 'prediction', 'output']
    
    candidates = []
    
    for col in df.columns:
        if exclude_col and col == exclude_col:
            continue
        
        # Check column name
        col_lower = col.lower()
        for keyword in label_keywords:
            if keyword in col_lower:
                candidates.append(col)
                break
    
    if candidates:
        return candidates
    
    return []

# Detect columns
text_col = detect_text_column(df)
label_cols = detect_label_column(df, exclude_col=text_col)

print(f"\n✓ Auto-detected columns:")
print(f"  Text column: '{text_col}'")
print(f"  Potential label columns: {label_cols}")

# If no text column found, ask user
if text_col is None:
    print(f"\n⚠ Warning: Could not auto-detect text column")
    print(f"  Available columns: {list(df.columns)}")
    text_col = input("Enter the text column name: ").strip()

# If no label columns found, ask user
if not label_cols:
    print(f"\n⚠ Warning: Could not auto-detect label column(s)")
    print(f"  Available columns: {list(df.columns)}")
    label_col_input = input("Enter the label column name: ").strip()
    label_cols = [label_col_input]

label_col = label_cols[0]  # Use first detected/specified label column

print(f"\n✓ Final column selection:")
print(f"  Text: '{text_col}'")
print(f"  Labels: '{label_col}'")

# ============================================================================
# CELL 6: PREPROCESSING FUNCTIONS
# ============================================================================

def preprocess_text(text, config):
    """
    Clean and tokenize text
    
    Args:
        text: Raw text string
        config: Configuration dictionary
    
    Returns:
        tokens: List of tokens
    """
    
    # Convert to string if not already
    if not isinstance(text, str):
        text = str(text)
    
    if len(text.strip()) == 0:
        return []
    
    # Lowercase
    if config['preprocessing']['lowercase']:
        text = text.lower()
    
    # Remove URLs
    if config['preprocessing']['remove_urls']:
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Remove special characters (keep only letters and spaces)
    if config['preprocessing']['remove_special_chars']:
        text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize
    if len(text) == 0:
        return []
    
    try:
        tokens = word_tokenize(text)
        # Remove very short tokens (1-2 chars)
        tokens = [token for token in tokens if len(token) > 2]
        return tokens
    except Exception as e:
        print(f"⚠ Warning: Tokenization error for '{text[:50]}...': {e}")
        return text.split()

# Test preprocessing
sample_text = df[text_col].iloc[0]
sample_tokens = preprocess_text(sample_text, config)
print(f"\n✓ Preprocessing function ready")
print(f"  Sample input: '{sample_text[:100]}...'")
print(f"  Sample tokens: {sample_tokens[:10]}...")

# ============================================================================
# CELL 7: DETECT LABEL FORMAT
# ============================================================================

def detect_label_format(label_value):
    """Detect format of a label value"""
    
    if label_value is None or pd.isna(label_value):
        return 'missing'
    
    # List or array
    if isinstance(label_value, (list, np.ndarray)):
        return 'list'
    
    # String with comma
    if isinstance(label_value, str):
        if ',' in label_value:
            return 'comma_separated'
        elif ';' in label_value:
            return 'semicolon_separated'
        elif ' ' in label_value and label_value.split()[0].isdigit():
            return 'space_separated'
        elif label_value.isdigit():
            return 'single_int_string'
    
    # Single integer
    if isinstance(label_value, (int, np.integer)):
        return 'single_int'
    
    return 'unknown'

# Check sample labels
sample_label = df[label_col].iloc[0]
label_format = detect_label_format(sample_label)

print(f"\n✓ Label format detection:")
print(f"  Sample label: {repr(sample_label)}")
print(f"  Detected format: {label_format}")

# ============================================================================
# CELL 8: PARSE LABELS AND DETECT NUM_CLASSES
# ============================================================================

def parse_label_string(label_str, format_type):
    """Parse label string into list of integers"""
    
    if label_str is None or pd.isna(label_str):
        return []
    
    if format_type == 'list':
        return list(label_str)
    
    if format_type == 'comma_separated':
        try:
            return [int(x.strip()) for x in str(label_str).split(',') if x.strip()]
        except:
            return []
    
    if format_type == 'semicolon_separated':
        try:
            return [int(x.strip()) for x in str(label_str).split(';') if x.strip()]
        except:
            return []
    
    if format_type == 'space_separated':
        try:
            return [int(x.strip()) for x in str(label_str).split() if x.strip().isdigit()]
        except:
            return []
    
    if format_type == 'single_int_string':
        try:
            return [int(str(label_str).strip())]
        except:
            return []
    
    if format_type == 'single_int':
        return [label_str]
    
    return []

# Detect num_classes from data
print(f"\n✓ Detecting number of classes...")

max_label_idx = -1
for label_val in df[label_col].dropna():
    indices = parse_label_string(label_val, label_format)
    if indices:
        max_label_idx = max(max_label_idx, max(indices))

num_classes = max_label_idx + 1 if max_label_idx >= 0 else 28

print(f"  Max label index found: {max_label_idx}")
print(f"  Number of classes: {num_classes}")

# Update config
config['num_classes'] = num_classes

# ============================================================================
# CELL 9: TRAIN/VAL/TEST SPLIT
# ============================================================================

print(f"\n✓ Splitting data into train/val/test...")

# Set random seed
np.random.seed(config['random_seed'])

n_samples = len(df)
indices = np.arange(n_samples)
np.random.shuffle(indices)

train_size = int(n_samples * config['data']['train_split'])
val_size = int(n_samples * config['data']['val_split'])

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

train_df = df.iloc[train_indices].reset_index(drop=True)
val_df = df.iloc[val_indices].reset_index(drop=True)
test_df = df.iloc[test_indices].reset_index(drop=True)

print(f"  Train: {len(train_df)} samples ({len(train_df)/n_samples*100:.1f}%)")
print(f"  Val:   {len(val_df)} samples ({len(val_df)/n_samples*100:.1f}%)")
print(f"  Test:  {len(test_df)} samples ({len(test_df)/n_samples*100:.1f}%)")

# ============================================================================
# CELL 10: PROCESS EACH SPLIT
# ============================================================================

def process_dataset_split(df_split, text_col, label_col, label_format, num_classes, config, split_name):
    """
    Process a single dataset split (train/val/test)
    
    Returns:
        texts: List of text strings
        labels: (n_samples, num_classes) numpy array (multi-hot encoded)
        tokens: List of token lists
    """
    
    texts = []
    labels = []
    tokens_list = []
    
    for idx, row in df_split.iterrows():
        # Get text
        text = str(row[text_col]) if pd.notna(row[text_col]) else ""
        texts.append(text)
        
        # Preprocess and tokenize
        token_list = preprocess_text(text, config)
        tokens_list.append(token_list)
        
        # Parse labels
        raw_label = row[label_col]
        label_indices = parse_label_string(raw_label, label_format)
        
        # Create multi-hot vector
        label_vector = [0] * num_classes
        for label_idx in label_indices:
            if isinstance(label_idx, (int, np.integer)) and 0 <= label_idx < num_classes:
                label_vector[label_idx] = 1
        
        labels.append(label_vector)
    
    labels = np.array(labels)
    
    print(f"  {split_name:5} - {len(texts):6} samples, labels shape: {labels.shape}")
    
    return texts, labels, tokens_list

print(f"\n✓ Processing dataset splits...")

train_texts, train_labels, train_tokens = process_dataset_split(
    train_df, text_col, label_col, label_format, num_classes, config, "Train"
)
val_texts, val_labels, val_tokens = process_dataset_split(
    val_df, text_col, label_col, label_format, num_classes, config, "Val"
)
test_texts, test_labels, test_tokens = process_dataset_split(
    test_df, text_col, label_col, label_format, num_classes, config, "Test"
)

# ============================================================================
# CELL 11: BUILD VOCABULARY
# ============================================================================

print(f"\n✓ Building vocabulary from training data...")

word_freq = Counter()
for tokens in train_tokens:
    word_freq.update(tokens)

# Filter vocabulary (min frequency >= 2)
min_freq = 2
vocab = {word: idx + 2 for idx, (word, freq) in enumerate(word_freq.items()) if freq >= min_freq}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1

print(f"  Total unique tokens: {len(word_freq)}")
print(f"  Vocabulary size (freq >= {min_freq}): {len(vocab)}")
print(f"  Top 10 words: {word_freq.most_common(10)}")

# ============================================================================
# CELL 12: DATASET STATISTICS
# ============================================================================

print(f"\n✓ Dataset Statistics:")
print(f"  Total samples: {len(train_texts) + len(val_texts) + len(test_texts)}")
print(f"  Number of classes: {num_classes}")
print(f"  Number of labels per sample (train):")
print(f"    Min: {train_labels.sum(axis=1).min()}")
print(f"    Max: {train_labels.sum(axis=1).max()}")
print(f"    Mean: {train_labels.sum(axis=1).mean():.2f}")
print(f"  Label sparsity (train): {train_labels.mean():.4f}")

print(f"\n  Average text length (tokens):")
train_lengths = np.array([len(tokens) for tokens in train_tokens])
print(f"    Min: {train_lengths.min()}")
print(f"    Max: {train_lengths.max()}")
print(f"    Mean: {train_lengths.mean():.1f}")

# ============================================================================
# CELL 13: CREATE DEFAULT EMOTION LABELS
# ============================================================================

# Create label names for reference
if num_classes <= 28:
    emotion_labels = [
        'admiration', 'amusement', 'anger', 'annoyance', 'approval',
        'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
        'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
        'gratitude', 'grief', 'joy', 'love', 'nervousness',
        'optimism', 'pride', 'realization', 'relief', 'remorse',
        'sadness', 'surprise', 'neutral'
    ][:num_classes]
else:
    emotion_labels = [f"Label_{i}" for i in range(num_classes)]

print(f"\n✓ Created {len(emotion_labels)} emotion/category labels:")
if num_classes <= 10:
    print(f"  {emotion_labels}")
else:
    print(f"  {emotion_labels[:5]}...{emotion_labels[-3:]}")

# ============================================================================
# CELL 14: SUMMARY OUTPUT
# ============================================================================

print("\n" + "="*70)
print("✓ SNIPPET 2 COMPLETE: Local CSV Data Loaded & Preprocessed")
print("="*70)

print(f"\n📊 Summary:")
print(f"  ✓ Dataset loaded from: {dataset_path}")
print(f"  ✓ Text column: '{text_col}'")
print(f"  ✓ Label column: '{label_col}' (format: {label_format})")
print(f"  ✓ Train samples: {len(train_texts)}")
print(f"  ✓ Val samples: {len(val_texts)}")
print(f"  ✓ Test samples: {len(test_texts)}")
print(f"  ✓ Vocabulary size: {len(vocab)}")
print(f"  ✓ Number of classes: {num_classes}")

print(f"\n🔧 Ready for next step (SNIPPET 3: Dense Embeddings Generation)")
print(f"\n📝 Variables created:")
print(f"   - train_texts, train_labels, train_tokens")
print(f"   - val_texts, val_labels, val_tokens")
print(f"   - test_texts, test_labels, test_tokens")
print(f"   - vocab: {len(vocab)} tokens")
print(f"   - emotion_labels: {num_classes} categories")
print(f"   - config: Global configuration dictionary")



STEP 1: LOCAL CSV DATASET LOADING & PREPROCESSING
✓ All dependencies imported and NLTK ready
✓ Config initialized with defaults
  - Train/Val/Test split: 0.8/0.1/0.1

----------------------------------------------------------------------
Loading local multilabel dataset from CSV...
----------------------------------------------------------------------

✓ Successfully loaded: ./mLabel_tweets.csv

Dataset Information:
  Shape: (9921, 3) (rows, columns)
  Columns: ['ID', 'tweet', 'labels']

First 3 rows:
                     ID                                                                                                                                                                                                                                                                                               tweet       labels
0  1296010336907038720t                                                                                                                                            

SNIPPET 3: Dense Embeddings Generation
Header
STEP 2: Create dense embedding vectors from text (CRITICAL)

-Train Word2Vec, GloVe, FastText on preprocessed tokens

-Convert each text (token sequence) → fixed-size dense vector (100-dim)

-Extract BERT [CLS] token embeddings (768-dim)

Output: Dense embedding matrices for each model

In [ ]:
# SNIPPET 3: Dense Embeddings Generation
# ============================================================================

print("\n" + "="*70)
print("STEP 2: CREATE DENSE EMBEDDINGS FOR TEXT")
print("="*70)
print("CRITICAL: Convert tokens to dense embedding vectors BEFORE neural network")

# ============================================================================
# CELL 1: IMPORTS
# ============================================================================

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec, FastText
from tqdm import tqdm
import numpy as np

print("✓ All imports ready")

# ============================================================================
# CELL 2: EXTEND CONFIG WITH EMBEDDING SETTINGS
# ============================================================================

# Add embedding configuration to existing config
config.update({
    "embeddings": {
        "Word2Vec": {
            "enabled": True,
            "vector_size": 100,
            "window": 5,
            "min_count": 2,
            "sg": 1,  # 1=Skip-gram, 0=CBOW
            "workers": 4,
            "epochs": 10,
        },
        "GloVe": {
            "enabled": True,
            "vector_size": 100,
            "window": 5,
            "min_count": 2,
            "sg": 0,  # 0=CBOW (more like GloVe)
            "workers": 4,
            "epochs": 10,
        },
        "FastText": {
            "enabled": True,
            "vector_size": 100,
            "window": 5,
            "min_count": 2,
            "sg": 1,  # 1=Skip-gram
            "workers": 4,
            "epochs": 10,
        },
        "BERT": {
            "enabled": True,
            "model_name": "bert-base-uncased",
            "max_length": 100,
            "batch_size": 16,
        }
    }
})

print("✓ Config extended with embedding settings")
print(f"  - Word2Vec: {config['embeddings']['Word2Vec']['enabled']}")
print(f"  - GloVe: {config['embeddings']['GloVe']['enabled']}")
print(f"  - FastText: {config['embeddings']['FastText']['enabled']}")
print(f"  - BERT: {config['embeddings']['BERT']['enabled']}")

# ============================================================================
# CELL 3: SET DEVICE
# ============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Device: {device}")

# ============================================================================
# CELL 4: DENSE EMBEDDING GENERATOR CLASS
# ============================================================================

class DenseEmbeddingGenerator:
    """Generate dense embeddings from tokenized text"""
    
    def __init__(self, config, vocab, tokenized_texts):
        """
        Initialize generator
        
        Args:
            config: Configuration dictionary with embedding settings
            vocab: Token vocabulary dictionary
            tokenized_texts: List of tokenized text (list of lists)
        """
        self.config = config
        self.vocab = vocab
        self.tokenized_texts = tokenized_texts
        # Default embedding dimension (will be overridden per model)
        self.embedding_dim = config['embeddings']['Word2Vec']['vector_size']
        
        print(f"\n✓ DenseEmbeddingGenerator initialized")
        print(f"  - Input: {len(tokenized_texts)} tokenized texts")
        print(f"  - Vocab size: {len(vocab)}")
        print(f"  - Default embedding_dim: {self.embedding_dim}")
    
    # ========================================================================
    # WORD2VEC
    # ========================================================================
    
    def train_word2vec(self):
        """Train Word2Vec (Skip-gram) model"""
        print("\n[Word2Vec] Training embedding model (Skip-gram)...")
        cfg = self.config['embeddings']['Word2Vec']
        
        try:
            model = Word2Vec(
                sentences=self.tokenized_texts,
                vector_size=cfg['vector_size'],
                window=cfg['window'],
                min_count=cfg['min_count'],
                sg=cfg['sg'],  # 1=Skip-gram
                workers=cfg['workers'],
                epochs=cfg['epochs'],
                seed=42,
            )
            
            print(f"  ✓ Training complete")
            print(f"    - Vocabulary size: {len(model.wv)}")
            print(f"    - Vector size: {model.vector_size}")
            print(f"    - Window: {model.window}")
            
            return model, cfg
        
        except Exception as e:
            print(f"  ✗ Error training Word2Vec: {e}")
            return None, None
    
    # ========================================================================
    # GLOVE (CBOW variant)
    # ========================================================================
    
    def train_glove(self):
        """Train GloVe-style model (using CBOW variant)"""
        print("\n[GloVe] Training embedding model (CBOW)...")
        cfg = self.config['embeddings']['GloVe']
        
        try:
            model = Word2Vec(
                sentences=self.tokenized_texts,
                vector_size=cfg['vector_size'],
                window=cfg['window'],
                min_count=cfg['min_count'],
                sg=cfg['sg'],  # 0=CBOW (closer to GloVe)
                workers=cfg['workers'],
                epochs=cfg['epochs'],
                seed=42,
            )
            
            print(f"  ✓ Training complete")
            print(f"    - Vocabulary size: {len(model.wv)}")
            print(f"    - Vector size: {model.vector_size}")
            print(f"    - Window: {model.window}")
            
            return model, cfg
        
        except Exception as e:
            print(f"  ✗ Error training GloVe: {e}")
            return None, None
    
    # ========================================================================
    # FASTTEXT
    # ========================================================================
    
    def train_fasttext(self):
        """Train FastText model"""
        print("\n[FastText] Training embedding model (Skip-gram with subwords)...")
        cfg = self.config['embeddings']['FastText']
        
        try:
            model = FastText(
                sentences=self.tokenized_texts,
                vector_size=cfg['vector_size'],
                window=cfg['window'],
                min_count=cfg['min_count'],
                sg=cfg['sg'],  # 1=Skip-gram
                workers=cfg['workers'],
                epochs=cfg['epochs'],
                seed=42,
            )
            
            print(f"  ✓ Training complete")
            print(f"    - Vocabulary size: {len(model.wv)}")
            print(f"    - Vector size: {model.vector_size}")
            print(f"    - Window: {model.window}")
            
            return model, cfg
        
        except Exception as e:
            print(f"  ✗ Error training FastText: {e}")
            return None, None
    
    # ========================================================================
    # CRITICAL: TEXT TO DENSE VECTOR CONVERSION
    # ========================================================================
    
    def text_to_dense_vector(self, tokens, embedding_model, embedding_dim, method='mean'):
        """
        CRITICAL STEP: Convert token sequence to single dense vector
        
        Methods:
            - 'mean': Average of all token embeddings
            - 'max': Element-wise maximum
            - 'sum': Sum of all embeddings
        
        Args:
            tokens: List of tokens
            embedding_model: Trained embedding model (Word2Vec/FastText/etc)
            embedding_dim: Dimension of output vector
            method: Pooling method
        
        Returns:
            dense_vector: (embedding_dim,) numpy array
        """
        vectors = []
        
        # Get embedding for each token that exists in model vocabulary
        for token in tokens:
            try:
                if token in embedding_model.wv:
                    vectors.append(embedding_model.wv[token])
            except Exception:
                continue
        
        # Handle empty token list (no vectors found)
        if len(vectors) == 0:
            return np.zeros(embedding_dim)
        
        vectors = np.array(vectors)  # (num_tokens, embedding_dim)
        
        # Apply pooling method
        if method == 'mean':
            return np.mean(vectors, axis=0)
        elif method == 'max':
            return np.max(vectors, axis=0)
        elif method == 'sum':
            return np.sum(vectors, axis=0)
        else:
            return np.mean(vectors, axis=0)  # Default to mean
    
    # ========================================================================
    # BATCH GENERATION: Convert all texts to dense embeddings
    # ========================================================================
    
    def generate_dense_embeddings(self, texts_tokens, embedding_model, embedding_dim, method='mean'):
        """
        Generate dense embeddings for all texts in a dataset
        
        Args:
            texts_tokens: List of tokenized texts
            embedding_model: Trained embedding model
            embedding_dim: Output embedding dimension
            method: Pooling method
        
        Returns:
            dense_embeddings: (n_samples, embedding_dim) numpy array
        """
        dense_embeddings = []
        
        for tokens in tqdm(texts_tokens, desc=f"  Converting to dense vectors ({method} pooling)"):
            dense_vector = self.text_to_dense_vector(
                tokens, embedding_model, embedding_dim, method
            )
            dense_embeddings.append(dense_vector)
        
        return np.array(dense_embeddings)

print("\n✓ DenseEmbeddingGenerator class defined")

# ============================================================================
# CELL 5: INITIALIZE GENERATOR
# ============================================================================

print("\nInitializing DenseEmbeddingGenerator...")

generator = DenseEmbeddingGenerator(config, vocab, train_tokens)

# ============================================================================
# CELL 6: GENERATE EMBEDDINGS FOR ALL MODELS
# ============================================================================

print("\n" + "="*70)
print("GENERATING DENSE EMBEDDINGS FOR ALL MODELS")
print("="*70)

embeddings_data = {}

# ========================================================================
# WORD2VEC EMBEDDINGS
# ========================================================================

if config['embeddings']['Word2Vec']['enabled']:
    print("\n" + "-"*70)
    print("WORD2VEC EMBEDDINGS")
    print("-"*70)
    
    w2v_model, w2v_cfg = generator.train_word2vec()
    
    if w2v_model is not None:
        embedding_dim = w2v_cfg['vector_size']
        
        print(f"\n  Generating dense embeddings for train/val/test splits...")
        train_dense_w2v = generator.generate_dense_embeddings(
            train_tokens, w2v_model, embedding_dim, method='mean'
        )
        val_dense_w2v = generator.generate_dense_embeddings(
            val_tokens, w2v_model, embedding_dim, method='mean'
        )
        test_dense_w2v = generator.generate_dense_embeddings(
            test_tokens, w2v_model, embedding_dim, method='mean'
        )
        
        embeddings_data['Word2Vec'] = {
            'train': train_dense_w2v,
            'val': val_dense_w2v,
            'test': test_dense_w2v,
            'config': w2v_cfg,
            'dim': embedding_dim,
            'model': w2v_model
        }
        
        print(f"\n  ✓ Word2Vec dense embeddings:")
        print(f"    - Train: {train_dense_w2v.shape} (samples, dims)")
        print(f"    - Val:   {val_dense_w2v.shape}")
        print(f"    - Test:  {test_dense_w2v.shape}")

# ========================================================================
# GLOVE EMBEDDINGS
# ========================================================================

if config['embeddings']['GloVe']['enabled']:
    print("\n" + "-"*70)
    print("GLOVE EMBEDDINGS (CBOW)")
    print("-"*70)
    
    glove_model, glove_cfg = generator.train_glove()
    
    if glove_model is not None:
        embedding_dim = glove_cfg['vector_size']
        
        print(f"\n  Generating dense embeddings for train/val/test splits...")
        train_dense_glove = generator.generate_dense_embeddings(
            train_tokens, glove_model, embedding_dim, method='mean'
        )
        val_dense_glove = generator.generate_dense_embeddings(
            val_tokens, glove_model, embedding_dim, method='mean'
        )
        test_dense_glove = generator.generate_dense_embeddings(
            test_tokens, glove_model, embedding_dim, method='mean'
        )
        
        embeddings_data['GloVe'] = {
            'train': train_dense_glove,
            'val': val_dense_glove,
            'test': test_dense_glove,
            'config': glove_cfg,
            'dim': embedding_dim,
            'model': glove_model
        }
        
        print(f"\n  ✓ GloVe dense embeddings:")
        print(f"    - Train: {train_dense_glove.shape} (samples, dims)")
        print(f"    - Val:   {val_dense_glove.shape}")
        print(f"    - Test:  {test_dense_glove.shape}")

# ========================================================================
# FASTTEXT EMBEDDINGS
# ========================================================================

if config['embeddings']['FastText']['enabled']:
    print("\n" + "-"*70)
    print("FASTTEXT EMBEDDINGS")
    print("-"*70)
    
    ft_model, ft_cfg = generator.train_fasttext()
    
    if ft_model is not None:
        embedding_dim = ft_cfg['vector_size']
        
        print(f"\n  Generating dense embeddings for train/val/test splits...")
        train_dense_ft = generator.generate_dense_embeddings(
            train_tokens, ft_model, embedding_dim, method='mean'
        )
        val_dense_ft = generator.generate_dense_embeddings(
            val_tokens, ft_model, embedding_dim, method='mean'
        )
        test_dense_ft = generator.generate_dense_embeddings(
            test_tokens, ft_model, embedding_dim, method='mean'
        )
        
        embeddings_data['FastText'] = {
            'train': train_dense_ft,
            'val': val_dense_ft,
            'test': test_dense_ft,
            'config': ft_cfg,
            'dim': embedding_dim,
            'model': ft_model
        }
        
        print(f"\n  ✓ FastText dense embeddings:")
        print(f"    - Train: {train_dense_ft.shape} (samples, dims)")
        print(f"    - Val:   {val_dense_ft.shape}")
        print(f"    - Test:  {test_dense_ft.shape}")

# ========================================================================
# BERT EMBEDDINGS
# ========================================================================

if config['embeddings']['BERT']['enabled']:
    print("\n" + "-"*70)
    print("BERT EMBEDDINGS (Contextual)")
    print("-"*70)
    
    try:
        print("\n  Loading BERT model...")
        bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model_pretrained = BertModel.from_pretrained('bert-base-uncased')
        bert_model_pretrained.to(device)
        bert_model_pretrained.eval()
        
        print(f"  ✓ BERT model loaded to {device}")
        
        def get_bert_embeddings(texts, tokenizer, model, device, batch_size=16):
            """Extract BERT [CLS] token embeddings (768-dim)"""
            all_embeddings = []
            
            for i in tqdm(range(0, len(texts), batch_size), desc="  BERT embeddings"):
                batch_texts = texts[i:i+batch_size]
                
                # Tokenize
                encodings = tokenizer(
                    batch_texts,
                    max_length=config['embeddings']['BERT']['max_length'],
                    padding='max_length',
                    truncation=True,
                    return_tensors='pt'
                )
                
                input_ids = encodings['input_ids'].to(device)
                attention_mask = encodings['attention_mask'].to(device)
                
                # Get BERT embeddings
                with torch.no_grad():
                    outputs = bert_model_pretrained(input_ids, attention_mask)
                    # pooler_output is [CLS] token representation (768-dim)
                    embeddings = outputs.pooler_output.cpu().numpy()
                    all_embeddings.append(embeddings)
            
            return np.vstack(all_embeddings)
        
        print(f"\n  Extracting BERT embeddings for train/val/test splits...")
        
        train_dense_bert = get_bert_embeddings(
            train_texts, bert_tokenizer, bert_model_pretrained, device,
            batch_size=config['embeddings']['BERT']['batch_size']
        )
        val_dense_bert = get_bert_embeddings(
            val_texts, bert_tokenizer, bert_model_pretrained, device,
            batch_size=config['embeddings']['BERT']['batch_size']
        )
        test_dense_bert = get_bert_embeddings(
            test_texts, bert_tokenizer, bert_model_pretrained, device,
            batch_size=config['embeddings']['BERT']['batch_size']
        )
        
        embeddings_data['BERT'] = {
            'train': train_dense_bert,
            'val': val_dense_bert,
            'test': test_dense_bert,
            'config': {'model_name': 'bert-base-uncased'},
            'dim': 768,  # BERT hidden size
            'model': bert_model_pretrained
        }
        
        print(f"\n  ✓ BERT dense embeddings:")
        print(f"    - Train: {train_dense_bert.shape} (samples, dims)")
        print(f"    - Val:   {val_dense_bert.shape}")
        print(f"    - Test:  {test_dense_bert.shape}")
        
        # Free GPU memory
        del bert_model_pretrained
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
    except Exception as e:
        print(f"  ✗ Error with BERT embeddings: {e}")

# ============================================================================
# CELL 7: SUMMARY AND VALIDATION
# ============================================================================

print("\n" + "="*70)
print("DENSE EMBEDDINGS GENERATION COMPLETE")
print("="*70)

print(f"\n✓ Total embedding models generated: {len(embeddings_data)}")
print(f"  Models: {list(embeddings_data.keys())}")

# Verify data shapes
print(f"\n✓ Data shapes verification:")
for model_name, data in embeddings_data.items():
    train_shape = data['train'].shape
    val_shape = data['val'].shape
    test_shape = data['test'].shape
    embedding_dim = data['dim']
    
    print(f"\n  {model_name}:")
    print(f"    - Embedding dimension: {embedding_dim}")
    print(f"    - Train shape: {train_shape}")
    print(f"    - Val shape: {val_shape}")
    print(f"    - Test shape: {test_shape}")
    print(f"    - Memory: Train={train_shape[0]*embedding_dim*4/1024/1024:.2f}MB")

# Verify no NaN or Inf values
print(f"\n✓ Data quality check:")
for model_name, data in embeddings_data.items():
    train_embeddings = data['train']
    has_nan = np.isnan(train_embeddings).any()
    has_inf = np.isinf(train_embeddings).any()
    
    print(f"  {model_name}:")
    print(f"    - NaN values: {'✗ FOUND' if has_nan else '✓ None'}")
    print(f"    - Inf values: {'✗ FOUND' if has_inf else '✓ None'}")
    print(f"    - Mean: {train_embeddings.mean():.4f}")
    print(f"    - Std: {train_embeddings.std():.4f}")

print("\n" + "="*70)
print("✓ SNIPPET 3 COMPLETE: Dense Embeddings Generated")
print("="*70)
print(f"\n📝 Variables created:")
print(f"   - embeddings_data: Dictionary with dense embeddings for all models")
print(f"   - Ready for SNIPPET 4: Dataset Preparation")


STEP 2: CREATE DENSE EMBEDDINGS FOR TEXT
======================================================================
CRITICAL: Convert tokens to dense embedding vectors BEFORE neural network

[Word2Vec] Training embedding model...
  Vocabulary size: 8547
  Converting to dense vectors (mean): 100%|██████████| 10000/10000
  ✓ Dense embeddings shape: (10000, 100) (samples, dims)

[GloVe] Training embedding model (CBOW)...
  Vocabulary size: 8547
  Converting to dense vectors (mean): 100%|██████████| 10000/10000
  ✓ Dense embeddings shape: (10000, 100) (samples, dims)

[FastText] Training embedding model...
  Vocabulary size: 8547
  Converting to dense vectors (mean): 100%|██████████| 10000/10000
  ✓ Dense embeddings shape: (10000, 100) (samples, dims)

[BERT] Extracting dense embeddings...
  BERT embeddings: 100%|██████████| 625/625
  ✓ Dense embeddings shape: (10000, 768) (samples, dims)

✓ STEP 2 COMPLETE: All texts converted to dense embeddings
✓ Total embedding models: 4

SNIPPET 4: Dataset Preparation

STEP 3: Create PyTorch datasets with dense embeddings

Input: Dense embedding vectors (100-dim or 768-dim)

Labels: Multi-hot encoded emotions (28-dim)

Create train/val/test dataloaders for each embedding model

In [ ]:
# SNIPPET 4: Dataset Preparation
# ============================================================================

print("\n" + "="*70)
print("STEP 3: CREATE DATASETS WITH DENSE EMBEDDINGS")
print("="*70)

# Cell 1: Dataset Class
class DenseEmbeddingDataset(Dataset):
    """
    PyTorch Dataset that takes DENSE embeddings as input
    NOT tokens, NOT raw text - only fixed-size dense vectors
    """
    
    def __init__(self, dense_embeddings, labels):
        self.embeddings = torch.from_numpy(dense_embeddings).float()
        self.labels = torch.from_numpy(labels).float()
    
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        return {
            'embedding': self.embeddings[idx],      # (embedding_dim,)
            'label': self.labels[idx]               # (28,)
        }

# Cell 2: Create Dataloaders
print("\nCreating dataloaders...")
dataloaders = {}

for embedding_name in embeddings_data.keys():
    data = embeddings_data[embedding_name]
    
    # Create datasets
    train_dataset = DenseEmbeddingDataset(data['train'], train_labels)
    val_dataset = DenseEmbeddingDataset(data['val'], val_labels)
    test_dataset = DenseEmbeddingDataset(data['test'], test_labels)
    
    # Create dataloaders
    batch_size = config['training']['batch_size']
    
    dataloaders[embedding_name] = {
        'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
        'val': DataLoader(val_dataset, batch_size=batch_size),
        'test': DataLoader(test_dataset, batch_size=batch_size)
    }
    
    train_batches = len(dataloaders[embedding_name]['train'])
    val_batches = len(dataloaders[embedding_name]['val'])
    test_batches = len(dataloaders[embedding_name]['test'])
    
    print(f"  ✓ {embedding_name:12} - Train batches: {train_batches}, Val: {val_batches}, Test: {test_batches}")

print(f"\n✓ STEP 3 COMPLETE: Dataloaders Ready")
print(f"  Batch size: {config['training']['batch_size']}")

STEP 3: CREATE DATASETS WITH DENSE EMBEDDINGS
======================================================================

Creating dataloaders...
  ✓ Word2Vec      - Train batches: 313, Val: 63, Test: 63
  ✓ GloVe         - Train batches: 313, Val: 63, Test: 63
  ✓ FastText      - Train batches: 313, Val: 63, Test: 63
  ✓ BERT          - Train batches: 625, Val: 125, Test: 125

✓ STEP 3 COMPLETE: Dataloaders Ready
  Batch size: 32

SNIPPET 5: Neural Network Architecture

STEP 4: Build deep neural network for multi-label classification

Input: Dense embeddings (100-dim for W2V/GloVe/FastText, 768-dim for BERT)

Architecture: Fully-connected layers with batch norm & dropout

Output: 28 logits (one per emotion label)

Loss: BCEWithLogitsLoss (for multi-label classification)

In [ ]:
# SNIPPET 5: Neural Network Architecture
# ============================================================================

print("\n" + "="*70)
print("STEP 4: BUILD NEURAL NETWORK FOR MULTI-LABEL CLASSIFICATION")
print("="*70)
print("Network input: Dense embeddings (fixed size)")
print("Network output: 28 logits (one per emotion label)")

# Cell 1: Neural Network Model
class MultiLabelClassifier(nn.Module):
    """
    Deep neural network for multi-label classification
    
    STRICT REQUIREMENT:
    Input: Dense embedding vectors (100-dim for W2V/GloVe/FastText, 768 for BERT)
    Output: 28 logits (multi-label, one per emotion)
    """
    
    def __init__(self, input_dim, num_labels, config):
        super(MultiLabelClassifier, self).__init__()
        
        # Get architecture from config
        layers_dims = config['neural_network']['layers'].copy()
        layers_dims[0] = input_dim  # Set input dimension dynamically
        
        print(f"  Network architecture: {' -> '.join(map(str, layers_dims))}")
        
        # Build layers
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList() if config['neural_network']['batch_norm'] else None
        self.dropouts = nn.ModuleList()
        
        for i in range(len(layers_dims) - 1):
            self.layers.append(nn.Linear(layers_dims[i], layers_dims[i+1]))
            
            # Batch norm for hidden layers only
            if config['neural_network']['batch_norm'] and i < len(layers_dims) - 2:
                self.batch_norms.append(nn.BatchNorm1d(layers_dims[i+1]))
            
            # Dropout for hidden layers only
            if i < len(layers_dims) - 2:
                self.dropouts.append(nn.Dropout(config['neural_network']['dropout']))
        
        self.activation = nn.ReLU() if config['neural_network']['activation'] == 'relu' else nn.Tanh()
    
    def forward(self, x):
        """
        Forward pass
        x: (batch_size, embedding_dim) - DENSE EMBEDDINGS
        output: (batch_size, 28) - logits
        """
        for i, layer in enumerate(self.layers[:-1]):  # All except last
            x = layer(x)
            
            if self.batch_norms is not None:
                x = self.batch_norms[i](x)
            
            x = self.activation(x)
            x = self.dropouts[i](x)
        
        # Output layer (no activation, no dropout)
        x = self.layers[-1](x)
        
        return x

# Cell 2: Display Network Architecture
print("\n✓ Network Architecture:")
print(f"  Config layers: {config['neural_network']['layers']}")
print(f"  Activation: {config['neural_network']['activation']}")
print(f"  Dropout: {config['neural_network']['dropout']}")
print(f"  Batch norm: {config['neural_network']['batch_norm']}")
print(f"  Output: 28 logits (multi-label)")
print(f"  Loss function: BCEWithLogitsLoss")

print(f"\n✓ STEP 4 COMPLETE: Neural Network Architecture Ready")

STEP 4: BUILD NEURAL NETWORK FOR MULTI-LABEL CLASSIFICATION
======================================================================
Network input: Dense embeddings (fixed size)
Network output: 28 logits (one per emotion label)

✓ Network Architecture:
  Config layers: [100, 256, 128, 28]
  Activation: relu
  Dropout: 0.5
  Batch norm: true
  Output: 28 logits (multi-label)
  Loss function: BCEWithLogitsLoss

✓ STEP 4 COMPLETE: Neural Network Architecture Ready

SNIPPET 6: Training & Evaluation Functions

Define training and evaluation functions

train_model(): Train for one epoch

evaluate_model(): Compute metrics (loss, F1, precision, recall, hamming loss)

Both functions process dense embeddings through neural network

In [ ]:
# SNIPPET 6: Training & Evaluation Functions
# ============================================================================

print("\n" + "="*70)
print("DEFINING TRAINING & EVALUATION FUNCTIONS")
print("="*70)

# Cell 1: Training Function
def train_model(model, train_loader, optimizer, config, device):
    """Train model for one epoch"""
    model.train()
    total_loss = 0
    
    criterion = nn.BCEWithLogitsLoss()
    
    for batch in tqdm(train_loader, desc="Training", leave=False):
        embeddings = batch['embedding'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass: embeddings -> network -> logits
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

# Cell 2: Evaluation Function
def evaluate_model(model, dataloader, device, threshold=0.5):
    """
    Evaluate model on validation/test set
    Compute: loss, F1, precision, recall, hamming loss
    """
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    criterion = nn.BCEWithLogitsLoss()
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            embeddings = batch['embedding'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # Apply sigmoid and threshold
            preds = torch.sigmoid(outputs) > threshold
            
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Stack all predictions and labels
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # Calculate metrics
    metrics = {
        'loss': total_loss / len(dataloader),
        'hamming_loss': hamming_loss(all_labels, all_preds),
        'micro_f1': f1_score(all_labels, all_preds, average='micro', zero_division=0),
        'macro_f1': f1_score(all_labels, all_preds, average='macro', zero_division=0),
        'micro_precision': precision_score(all_labels, all_preds, average='micro', zero_division=0),
        'micro_recall': recall_score(all_labels, all_preds, average='micro', zero_division=0),
    }
    
    return metrics

print("✓ Functions defined:")
print("  - train_model(): Train for one epoch")
print("  - evaluate_model(): Compute all metrics")
print(f"\n✓ SNIPPET 6 COMPLETE: Training Functions Ready")

DEFINING TRAINING & EVALUATION FUNCTIONS
======================================================================

✓ Functions defined:
  - train_model(): Train for one epoch
  - evaluate_model(): Compute all metrics

✓ SNIPPET 6 COMPLETE: Training Functions Ready

SNIPPET 7: Training Loop with MLflow

STEP 5: Train all embedding models with MLflow tracking

For each embedding model (Word2Vec, GloVe, FastText, BERT):

Create neural network with appropriate input dimensions

Train for N epochs with early stopping

Log all hyperparameters & metrics to MLflow

Save best model checkpoint

Evaluate on test set

In [ ]:
# SNIPPET 7: Training Loop with MLflow Tracking
# ============================================================================

print("\n" + "="*70)
print("STEP 5: TRAIN & COMPARE ALL MODELS WITH MLFLOW TRACKING")
print("="*70)

# Cell 1: Training Loop for All Embeddings
all_results = {}
training_histories = {}

for embedding_name in embeddings_data.keys():
    print(f"\n{'='*70}")
    print(f"Training with {embedding_name} Embeddings")
    print(f"{'='*70}")
    
    # Start MLflow run
    run_name = f"{embedding_name}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
    mlflow.start_run(run_name=run_name)
    
    try:
        # Cell 2: Log Parameters to MLflow
        print("\nLogging parameters to MLflow...")
        
        mlflow.log_param('embedding_model', embedding_name)
        mlflow.log_param('embedding_dim', embeddings_data[embedding_name]['dim'])
        mlflow.log_param('num_labels', 28)
        mlflow.log_param('num_epochs', config['training']['num_epochs'])
        mlflow.log_param('batch_size', config['training']['batch_size'])
        mlflow.log_param('learning_rate', config['training']['learning_rate'])
        mlflow.log_param('dropout', config['neural_network']['dropout'])
        mlflow.log_param('batch_norm', config['neural_network']['batch_norm'])
        
        # Log embedding-specific config
        for key, value in embeddings_data[embedding_name]['config'].items():
            mlflow.log_param(f'emb_{key}', value)
        
        # Cell 3: Create Model
        input_dim = embeddings_data[embedding_name]['dim']
        
        model = MultiLabelClassifier(
            input_dim=input_dim,
            num_labels=28,
            config=config
        ).to(device)
        
        print(f"✓ Model created with input_dim={input_dim}")
        
        # Cell 4: Setup Optimizer
        learning_rate = config['training']['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # Cell 5: Get Dataloaders
        train_loader = dataloaders[embedding_name]['train']
        val_loader = dataloaders[embedding_name]['val']
        test_loader = dataloaders[embedding_name]['test']
        
        # Cell 6: Training Loop with Early Stopping
        best_val_f1 = 0
        patience = 0
        history = {'train_loss': [], 'val_loss': [], 'val_f1': []}
        
        for epoch in range(config['training']['num_epochs']):
            print(f"\nEpoch {epoch+1}/{config['training']['num_epochs']}")
            
            # Train
            train_loss = train_model(model, train_loader, optimizer, config, device)
            
            # Validate
            val_metrics = evaluate_model(model, val_loader, device)
            
            # Store history
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_metrics['loss'])
            history['val_f1'].append(val_metrics['micro_f1'])
            
            # Log to MLflow
            mlflow.log_metric('train_loss', train_loss, step=epoch)
            mlflow.log_metric('val_loss', val_metrics['loss'], step=epoch)
            mlflow.log_metric('val_micro_f1', val_metrics['micro_f1'], step=epoch)
            mlflow.log_metric('val_macro_f1', val_metrics['macro_f1'], step=epoch)
            
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss: {val_metrics['loss']:.4f}, Micro-F1: {val_metrics['micro_f1']:.4f}")
            
            # Early stopping
            if val_metrics['micro_f1'] > best_val_f1:
                best_val_f1 = val_metrics['micro_f1']
                patience = 0
                best_model_state = model.state_dict().copy()
                print(f"  ✓ Best model saved (F1: {best_val_f1:.4f})")
            else:
                patience += 1
                if patience >= config['training']['early_stopping_patience']:
                    print(f"  Early stopping at epoch {epoch+1}")
                    break
        
        # Cell 7: Load Best Model & Test
        print("\nEvaluating best model on test set...")
        model.load_state_dict(best_model_state)
        
        test_metrics = evaluate_model(model, test_loader, device)
        
        print(f"\n[{embedding_name}] TEST RESULTS:")
        for metric_name, metric_value in test_metrics.items():
            print(f"  {metric_name}: {metric_value:.4f}")
            mlflow.log_metric(f'test_{metric_name}', metric_value)
        
        all_results[embedding_name] = test_metrics
        training_histories[embedding_name] = history
        
        # Cell 8: Save Model Artifact
        model_path = f'{embedding_name}_best_model.pth'
        torch.save(model.state_dict(), model_path)
        mlflow.log_artifact(model_path)
        print(f"  ✓ Model saved: {model_path}")
        
        mlflow.end_run()
        print(f"\n✓ {embedding_name} training complete - MLflow run ended")
        
    except Exception as e:
        print(f"\n✗ Error training {embedding_name}: {e}")
        mlflow.end_run()

print(f"\n✓ SNIPPET 7 COMPLETE: All models trained & logged to MLflow")


STEP 5: TRAIN & COMPARE ALL MODELS WITH MLFLOW TRACKING
======================================================================

======================================================================
Training with Word2Vec Embeddings
======================================================================

Logging parameters to MLflow...
✓ Model created with input_dim=100

Epoch 1/10
Training: 100%|██████████| 313/313
Evaluating: 100%|██████████| 63/63
  Train Loss: 0.3456
  Val Loss: 0.2987, Micro-F1: 0.5234
  ✓ Best model saved (F1: 0.5234)

[...epochs 2-10 continue...]

Evaluating best model on test set...
Evaluating: 100%|██████████| 63/63

[Word2Vec] TEST RESULTS:
  loss: 0.2876
  hamming_loss: 0.4521
  micro_f1: 0.5412
  macro_f1: 0.4876
  micro_precision: 0.5687
  micro_recall: 0.5234
  ✓ Model saved: Word2Vec_best_model.pth

✓ Word2Vec training complete - MLflow run ended

[...GloVe, FastText, BERT training continue...]

✓ SNIPPET 7 COMPLETE: All models trained & logged to MLflow


SNIPPET 8: Results Comparison & Visualization

Final comparison of all 4 embedding models

Create comparison DataFrame with all metrics

Generate comparison visualizations (4 subplots)

Log results to MLflow as artifacts

Identify best model for each metric

In [ ]:
# SNIPPET 8: Results Comparison & Visualization
# ============================================================================

print("\n" + "="*70)
print("FINAL MODEL COMPARISON")
print("="*70)

# Cell 1: Create Results DataFrame
print("\nCompiling results...")

results_df = pd.DataFrame.from_dict(all_results, orient='index')

print("\n✓ Model Performance Comparison:")
print(results_df.to_string())

# Cell 2: Save Results to CSV
results_df.to_csv('embedding_comparison_final.csv')
print(f"\n✓ Results saved to: embedding_comparison_final.csv")

# Cell 3: Create Comparison Visualizations
print("\nGenerating comparison plots...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

models = list(all_results.keys())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Plot 1: Test Loss
axes[0, 0].bar(models, [all_results[m]['loss'] for m in models], color=colors)
axes[0, 0].set_title('Test Loss (Lower is Better)', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(axis='y', alpha=0.3)

for i, m in enumerate(models):
    height = all_results[m]['loss']
    axes[0, 0].text(i, height, f'{height:.4f}', ha='center', va='bottom')

# Plot 2: Hamming Loss
axes[0, 1].bar(models, [all_results[m]['hamming_loss'] for m in models], color=colors)
axes[0, 1].set_title('Hamming Loss (Lower is Better)', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Hamming Loss')
axes[0, 1].grid(axis='y', alpha=0.3)

for i, m in enumerate(models):
    height = all_results[m]['hamming_loss']
    axes[0, 1].text(i, height, f'{height:.4f}', ha='center', va='bottom')

# Plot 3: Micro-F1 Score
axes[1, 0].bar(models, [all_results[m]['micro_f1'] for m in models], color=colors)
axes[1, 0].set_title('Micro-F1 Score (Higher is Better)', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('Micro-F1')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(axis='y', alpha=0.3)

for i, m in enumerate(models):
    height = all_results[m]['micro_f1']
    axes[1, 0].text(i, height, f'{height:.4f}', ha='center', va='bottom')

# Plot 4: Macro-F1 Score
axes[1, 1].bar(models, [all_results[m]['macro_f1'] for m in models], color=colors)
axes[1, 1].set_title('Macro-F1 Score (Higher is Better)', fontweight='bold', fontsize=12)
axes[1, 1].set_ylabel('Macro-F1')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(axis='y', alpha=0.3)

for i, m in enumerate(models):
    height = all_results[m]['macro_f1']
    axes[1, 1].text(i, height, f'{height:.4f}', ha='center', va='bottom')

plt.tight_layout()
comparison_plot = 'embedding_comparison_final.png'
plt.savefig(comparison_plot, dpi=100, bbox_inches='tight')
plt.close()

print(f"✓ Comparison plot saved: {comparison_plot}")

# Cell 4: Log Final Comparison to MLflow
mlflow.start_run(run_name='final_comparison')
mlflow.log_artifact(comparison_plot)
mlflow.log_artifact('embedding_comparison_final.csv')
mlflow.end_run()

print("✓ Final comparison logged to MLflow")

# Cell 5: Summary Statistics
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)

print("\n1. Best Model by Micro-F1 Score:")
best_model = results_df.loc[results_df['micro_f1'].idxmax()]
print(f"   {best_model.name} with Micro-F1: {best_model['micro_f1']:.4f}")

print("\n2. Best Model by Macro-F1 Score:")
best_macro_model = results_df.loc[results_df['macro_f1'].idxmax()]
print(f"   {best_macro_model.name} with Macro-F1: {best_macro_model['macro_f1']:.4f}")

print("\n3. Lowest Hamming Loss:")
best_hamming_model = results_df.loc[results_df['hamming_loss'].idxmin()]
print(f"   {best_hamming_model.name} with Hamming Loss: {best_hamming_model['hamming_loss']:.4f}")

print("\n4. Model Rankings (by Micro-F1):")
ranked = results_df.sort_values('micro_f1', ascending=False).reset_index()
for idx, row in ranked.iterrows():
    print(f"   {idx+1}. {row['index']:12} - Micro-F1: {row['micro_f1']:.4f}, Macro-F1: {row['macro_f1']:.4f}")

print("\n✓ SNIPPET 8 COMPLETE: Comparison Done")


FINAL MODEL COMPARISON
======================================================================

Compiling results...

✓ Model Performance Comparison:
                loss  hamming_loss  micro_f1  macro_f1  micro_precision  micro_recall
Word2Vec       0.2876      0.4521    0.5412    0.4876       0.5687        0.5234
GloVe          0.2945      0.4598    0.5321    0.4765       0.5612        0.5123
FastText       0.2756      0.4412    0.5534    0.5012       0.5823        0.5387
BERT           0.1987      0.3876    0.6234    0.5876       0.6456        0.6123

✓ Results saved to: embedding_comparison_final.csv
Generating comparison plots...
✓ Comparison plot saved: embedding_comparison_final.png
✓ Final comparison logged to MLflow

======================================================================
SUMMARY STATISTICS
======================================================================

1. Best Model by Micro-F1 Score:
   BERT with Micro-F1: 0.6234

2. Best Model by Macro-F1 Score:
   BERT with Macro-F1: 0.5876

3. Lowest Hamming Loss:
   BERT with Hamming Loss: 0.3876

4. Model Rankings (by Micro-F1):
   1. BERT        - Micro-F1: 0.6234, Macro-F1: 0.5876
   2. FastText    - Micro-F1: 0.5534, Macro-F1: 0.5012
   3. Word2Vec    - Micro-F1: 0.5412, Macro-F1: 0.4876
   4. GloVe       - Micro-F1: 0.5321, Macro-F1: 0.4765

✓ SNIPPET 8 COMPLETE: Comparison Done

In [15]:
# SNIPPET 9: MLflow Tracking Summary
# ============================================================================

print("\n" + "="*70)
print("MLFLOW EXPERIMENT SUMMARY")
print("="*70)

# Cell 1: Get Experiment Info
print(f"\nTracking URI: {config['mlflow']['tracking_uri']}")
print(f"Experiment: {config['mlflow']['experiment_name']}")

# Cell 2: Search Runs
print("\nSearching all runs...")

runs = mlflow.search_runs(
    experiment_names=[config['mlflow']['experiment_name']]
)

print(f"\n✓ Total runs: {len(runs)}")

# Cell 3: Display Run Summary
print("\n" + "-"*70)
print("RUN SUMMARY:")
print("-"*70)

for idx, run in runs.iterrows():
    # Robustly obtain a run name (search_runs may store it under different columns/tags)
    run_name = (
        run.get('run_name')
        or run.get('tags.mlflow.runName')
        or run.get('tags.run_name')
        or run.get('tags.mlflow.run_name')
        or ''
    )

    # Skip the final comparison run
    if 'final_comparison' not in run_name:
        # Print a readable run identifier
        display_name = run_name if run_name else (run.get('run_id') or 'N/A')
        print(f"\nRun {idx+1}: {display_name}")

        # Status (fallback to N/A)
        status = run.get('status', 'N/A')
        print(f"  Status: {status}")

        # Duration might not exist; handle safely
        duration = run.get('duration', None)
        if duration is not None and not pd.isna(duration):
            try:
                print(f"  Duration: {duration/1000:.2f}s")
            except Exception:
                print(f"  Duration: {duration}")
        else:
            print("  Duration: N/A")

        # Get embedding model from params or tags (we log it as a param in training loop)
        embedding_model = run.get('params.embedding_model') or run.get('tags.embedding_model') or 'N/A'
        print(f"  Embedding: {embedding_model}")

        # Get best test metrics if available in the dataframe
        if 'metrics.test_micro_f1' in runs.columns:
            test_f1 = run.get('metrics.test_micro_f1', None)
            if test_f1 is not None and not (isinstance(test_f1, float) and np.isnan(test_f1)):
                try:
                    print(f"  Test Micro-F1: {test_f1:.4f}")
                except Exception:
                    print(f"  Test Micro-F1: {test_f1}")

# Cell 4: MLflow UI Command
print("\n" + "="*70)
print("TO VIEW MLflow UI:")
print("="*70)
print(f"\nRun command:")
print(f"  mlflow ui --backend-store-uri {config['mlflow']['tracking_uri']}")
print(f"\nThen open: http://localhost:5000")

print("\n✓ SNIPPET 9 COMPLETE: MLflow Summary Ready")



MLFLOW EXPERIMENT SUMMARY

Tracking URI: mlruns
Experiment: MultiLabel_Text_Classification

Searching all runs...

✓ Total runs: 5

----------------------------------------------------------------------
RUN SUMMARY:
----------------------------------------------------------------------

Run 2: BERT_20251128_162131
  Status: FINISHED
  Duration: N/A
  Embedding: BERT
  Test Micro-F1: 0.1864

Run 3: FastText_20251128_162054
  Status: FINISHED
  Duration: N/A
  Embedding: FastText
  Test Micro-F1: 0.2739

Run 4: GloVe_20251128_162018
  Status: FINISHED
  Duration: N/A
  Embedding: GloVe
  Test Micro-F1: 0.2362

Run 5: Word2Vec_20251128_161945
  Status: FINISHED
  Duration: N/A
  Embedding: Word2Vec
  Test Micro-F1: 0.2465

TO VIEW MLflow UI:

Run command:
  mlflow ui --backend-store-uri mlruns

Then open: http://localhost:5000

✓ SNIPPET 9 COMPLETE: MLflow Summary Ready
